Requirements
numpy==1.19.5
pandas==1.0.1
scikit-learn==0.22.1
scipy==1.4.1
joblib==0.14.1
mlflow==1.6.0
matplotlib==3.7.0
scikit-learn==1.2.1


In [64]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from joblib import dump,load





In [65]:
df = pd.read_csv('../data.csv', delimiter = ',')
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [66]:
# Seleciona as variáveis independentes
independentcols = ['income', 'age', 'year', 'Credit_Score']

# Normaliza o campo age com numeros 
le = LabelEncoder()
df['age'] = le.fit_transform(df['age'])

# Dropa do dataset os valores nulos das variáveis indepententes
df = df.dropna(subset=independentcols)

# X recebe somente as variáveis independentes do treinamento do cluster
X = df[independentcols]


In [67]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139520 entries, 0 to 148669
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   income        139520 non-null  float64
 1   age           139520 non-null  int32  
 2   year          139520 non-null  int64  
 3   Credit_Score  139520 non-null  int64  
dtypes: float64(1), int32(1), int64(2)
memory usage: 4.8 MB


In [68]:
le

LabelEncoder()

In [69]:
# Normalizar as variáveis independentes
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [70]:
# Escolha o número de clusters
kmeans = KMeans(n_clusters=6)
kmeans.fit(X_norm)

C:\Users\leovi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=6)

In [71]:
# Adiciona o cluster como uma nova coluna no dataframe
df['Cluster'] = kmeans.labels_
X['Cluster'] = kmeans.labels_

C:\Users\leovi\AppData\Local\Temp\ipykernel_29220\3008842561.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cluster'] = kmeans.labels_


In [72]:
y = df['Status']

In [73]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=.3, random_state=10)

In [74]:

# Cria modelo de regressão logística para prever a inadimplência
X_cluster = pd.get_dummies(X_train['Cluster'], prefix='Cluster')
X_all = pd.concat([X_train, X_cluster], axis=1)
model = LogisticRegression()
model.fit(X_all, y_train)

LogisticRegression()

In [75]:
X_cluster_test = pd.get_dummies(X_test['Cluster'], prefix='Cluster')
X_all_test = pd.concat([X_test, X_cluster_test], axis=1)
y_pred = model.predict_proba(X_all_test)[:, 1]
# y_pred
print(len(y_test))
print(len(y_pred))


41856
41856


In [76]:
# Avalia o modelo
from sklearn.metrics import accuracy_score

auc = roc_auc_score(y_test, y_pred)
# acc = accuracy_score(y_test, y_pred)

print (y_pred)
print (y_test)

print('AUC:', auc)

[0.23492098 0.25200408 0.24919696 ... 0.26382682 0.24241176 0.22907925]
122888    1
155       1
66574     0
90773     1
113912    1
         ..
92638     0
22400     0
82690     0
78766     0
103365    0
Name: Status, Length: 41856, dtype: int64
AUC: 0.5914179571212089


In [77]:
# # Carrega o modelo
# dump(model,'modelo.joblib')

# Carrega o cluster
dump(kmeans,'cluster.joblib')

# Carrega o labelencoder
dump(le, 'labelencoder.joblib')

# Carrega o scaler de normalização
dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [79]:
print(le.transform(['55-64']))
print(le.inverse_transform([3]))


[3]
['55-64']
